### Лабораторная работа №1. Симплекс-метод

In [142]:
import numpy as np
import matplotlib.pyplot as plt
import json

Класс **Linear task** описывает задачу линейного программирования: коэффициенты оптимизируемой линейной функции и ограничения, а также предоставляет реализацию симплекс метода для решения задачи.

Данные о функции и ограничения считываются из json файла, в качестве результата работы симплекс метода предоставляется вектор значений аргументов.

In [239]:
class Linear_task:
    def __init__(self):
        self.func: np.array(int)
        self.goal: str
        self.constraints: list[dict()]
        self.constraints_matrix: np.array()
        self.b_array: np.array()
        self.original_size: int

        self.simplex_table = dict()
        self.optimal: np.array()
        self.basis: np.array()
        self.func_b = 0
    
    # считывание условий задачи из файла json
    def load_data_from_json(self, filename: str) -> None:
        with open(filename) as f:
            data = json.load(f)
        self.func = np.array(data[' f '].copy())
        self.original_size = len(self.func)
        self.goal = data[' goal ']
        self.constraints = data[' constraints ']
        self.constraints_matrix = np.array([line[' coefs '] for line in self.constraints])
        self.b_array = np.array([line[' b '] for line in self.constraints])
        self.make_canonic_form()

    # переход к канонической форме задачи - добавление балансирующих переменных
    def make_canonic_form(self) -> None:
        for index, line in enumerate(self.constraints):
            if line[' type '] != ' eq ':
                if line[' type '] == ' gte ':
                    self.b_array[index] = -self.b_array[index]
                    for item in range(len(self.constraints_matrix[index])):
                        self.constraints_matrix[index][item] = -self.constraints_matrix[index][item]
                self.constraints_matrix = np.append(self.constraints_matrix, 
                        [[1] if i == index else [0] for i in range(len(self.constraints_matrix))], axis= 1)
                self.func = np.append(self.func, 0)
            else:
                line_gte = self.constraints_matrix[index].copy()
                for i in range(len(line_gte)):
                    line_gte[i] = -line_gte[i]
                
                self.constraints_matrix = np.append(self.constraints_matrix, [line_gte], axis= 0)
                self.b_array = np.append(self.b_array, -self.b_array[index])
                index_gte = len(self.constraints_matrix) - 1
                self.constraints_matrix = np.append(self.constraints_matrix, 
                        [[1] if i == index else [0] for i in range(len(self.constraints_matrix))], axis= 1)
                self.func = np.append(self.func, 0)
                self.constraints_matrix = np.append(self.constraints_matrix, 
                        [[1] if i == index_gte else [0] for i in range(len(self.constraints_matrix))], axis= 1)
                self.func = np.append(self.func, 0)

        
    def simplex_method(self):
        self.set_start_values()

        while True:
            column = self.choose_solving_column()
            # остановка алгоритма
            if column == -1 or column == np.inf:
                break
            row, new_value = self.choose_solving_row(column)
            self.recount_matrix(column, row)
            self.recount_optimal(column, new_value, row)
            self.recount_simplex_table(column, row)
            self.change_func_coefs(column, row)
            print("Значение функции:    ", np.dot(self.optimal, self.func.T) + self.func_b)

        print("Решение:  ", self.optimal[: self.original_size])

    def change_func_coefs(self, index_new_basis, index_old_basis):

        for i in range(len(self.constraints_matrix)):
            if self.constraints_matrix[i][index_new_basis] != 0:
                self.func_b += self.func[index_new_basis]*self.b_array[i]
                for j, value in enumerate(self.constraints_matrix[i]):
                    if j != index_new_basis:
                        self.func[j] += -self.func[index_new_basis]*value
                        
        self.func[index_new_basis] = 0
            
    def recount_simplex_table(self, index_new_basis, index_old_basis):
        self.simplex_table[index_new_basis] = dict.fromkeys(list(self.simplex_table[index_old_basis].keys()))
        del self.simplex_table[index_old_basis]

        for key, value in self.simplex_table.items():
            del value[index_new_basis]
            for i in range(len(self.constraints_matrix)):
                if self.constraints_matrix[i][key] != 0:
                    value[index_old_basis] = self.constraints_matrix[i][index_old_basis]/self.constraints_matrix[i][key]
            
        for key, value in self.simplex_table[index_new_basis].items():
            if key == index_old_basis:
                continue
            for i in range(len(self.constraints_matrix)):
                if self.constraints_matrix[i][index_new_basis] == 0:
                    continue
                self.simplex_table[index_new_basis][key] = self.constraints_matrix[i][key]/self.constraints_matrix[i][index_new_basis]
    
    def recount_matrix(self, old_index, new_index):
        matrix = np.copy(self.constraints_matrix)
        basis_indices = list(self.simplex_table.keys())
        basis_indices.remove(new_index)
        b = np.copy(self.b_array)
        line_number = -1
        for row in range(len(self.constraints_matrix)):
            flag_right_line = True
            for k in basis_indices:
                if matrix[row][k] != 0:
                    flag_right_line = False
            if matrix[row][old_index] == 0:
                continue
            elif flag_right_line:
                line_number = row
                break

        for row in range(len(self.constraints_matrix)):
            if matrix[row][old_index] == 0 or row == line_number:
                continue
            else:
                f = matrix[row][old_index] / matrix[line_number][old_index]
                for j in range(len(self.constraints_matrix[row])):
                    matrix[row][j] -= f * matrix[line_number][j]
                b[row] -= f * b[line_number]
            
        self.constraints_matrix = matrix
        self.b_array = b

    def recount_optimal(self, index_new_basis, new_value, index_old_basis):
        for i in self.simplex_table.keys():
            for row in range(len(self.constraints_matrix)):
                if self.constraints_matrix[row][i] == 1:
                    self.optimal[i] = self.b_array[row]

        self.optimal[index_new_basis] = new_value
        self.optimal[index_old_basis] = 0

    def set_start_values(self):
        self.optimal = [0 for i in range(len(self.func))]
        self.simplex_table = dict.fromkeys([i for i in range(self.original_size, len(self.func))])
        
        for i in range(self.original_size, len(self.func)):
            line = i - self.original_size
            row = {j : self.constraints_matrix[line][j] for j in range(self.original_size)}
            self.simplex_table[i] = row.copy()
        
        self.optimal = np.append(([0 for i in range(self.original_size)]), ([self.b_array[j] for j in range(len(self.func) - self.original_size)]))
        self.optimal_func = np.dot(self.optimal, self.func.T)

    def choose_solving_row(self, column):
        dict_ = {key: self.b_array[key]/value[column] for key, value in self.simplex_table.items() if value[column] != 0}
        min_value = min(list(dict_.values()))
        print(min_value, list(dict_.values()))
        row = list(dict_.keys())[list(dict_.values()).index(min_value)]

        return row, min_value

    def choose_solving_column(self):
        max_index = -1
        min_index = -1
        for i in list(self.simplex_table[list(self.simplex_table.keys())[0]].keys()):
            if self.goal == " max ":
                if self.func[i] >= self.func[max_index] and self.func[i] >= 0:
                    max_index = i
            elif self.goal == " min ":
                if self.func[i] <= self.func[min_index] and self.func[i] <= 0:
                    min_index = i
        
        if self.goal == " max ":
            return max_index
        elif self.goal == " min ":
            return min_index

In [241]:
task = Linear_task()
task.load_data_from_json('data.json')
print(task.constraints_matrix, task.b_array)
task.set_start_values()
#task.simplex_method()
print(task.simplex_table)
#col = task.choose_solving_column()
#row, val = task.choose_solving_row(col)
print(task.constraints_matrix, task.b_array)
#task.set_start_values()
#col = task.choose_solving_column()
#row, val = task.choose_solving_row(col)
#print(row, col)
#task.recount_matrix(col, row)
#print(task.constraints_matrix, task.b_array)
# task.recount_optimal(col, val, row)
# task.recount_simplex_table(col, row)
# task.change_func_coefs(col, row)

[[ 1  0  0  1  0  0  0]
 [-1 -1  0  0  1  0  0]
 [ 1  1  1  0  0  1  0]
 [-1 -1 -1  0  0  0  1]] [ 1 -2  3 -3]
{3: {0: 1, 1: 0, 2: 0}, 4: {0: -1, 1: -1, 2: 0}, 5: {0: 1, 1: 1, 2: 1}, 6: {0: -1, 1: -1, 2: -1}}
[[ 1  0  0  1  0  0  0]
 [-1 -1  0  0  1  0  0]
 [ 1  1  1  0  0  1  0]
 [-1 -1 -1  0  0  0  1]] [ 1 -2  3 -3]


In [93]:
def recount_matrix(self, old_index, new_index):
        matrix = np.copy(self.constraints_matrix)
        basis_indices = list(self.simplex_table.keys())
        b = np.copy(self.b_array)
        for row in range(len(self.constraints_matrix)):
            if matrix[row][index] == 0 or index == row:
                continue
            else:
                f = matrix[row][index] / matrix[index][index]
                for j in range(len(self.constraints_matrix)):
                    matrix[row][j] -= f * matrix[index][j]
                b[row] -= f * b[index]

        self.constraints_matrix = matrix
        self.b_array = b


    def set_start_values(self):
        self.optimal = [0 for i in range(len(self.func))]
        for i in range(self.original_size, len(self.func)):
            line = i - self.original_size
            row = {j : self.constraints_matrix[line][j] for j in range(self.original_size)}
            # /self.constraints_matrix[line][i]
            if self.simplex_table is None:
                self.simplex_table = {i: row.copy()}
            else:
                self.simplex_table.update({i: row.copy()})
            
        for i in range(self.original_size):
            row = self.constraints_matrix[i]
            if 0 in row[:self.original_size]:
                self.optimal[i] = 0
                for j in range(self.original_size, len(self.func)):
                    if row[j] == 1:
                        self.optimal[j] = self.b_array[i]
            else:
                self.optimal[i] = self.b_array[i]
        
        self.optimal = np.array(self.optimal)
        #np.append(([0 for i in range(self.original_size)]), ([self.b_array[j] for j in range(len(self.func) - self.original_size)]))
        self.optimal_func = np.dot(self.optimal, self.func.T)

In [157]:
task = Linear_task()
task.load_data_from_json('data.json')
print(task.constraints_matrix)
task.set_start_values()
print(task.simplex_table)

col = task.choose_solving_column()
print(col)
row, val = task.choose_solving_row(col)

task.recount_matrix(col, row)
print(task.constraints_matrix)

[[1 0 0 1 0 0 0]
 [1 1 0 0 1 0 0]
 [1 1 1 0 0 1 0]
 [1 1 1 0 0 0 1]]
{3: {0: 1, 1: 0, 2: 0}, 4: {0: 1, 1: 1, 2: 0}, 5: {0: 1, 1: 1, 2: 1}, 6: {0: 1, 1: 1, 2: 1}}
2
[[1 0 0 1 0 0 0]
 [1 1 0 0 1 0 0]
 [1 1 1 0 0 1 0]
 [0 0 0 0 0 0 1]]


In [84]:
task = Linear_task()
task.load_data_from_json('data_2.json')
# print(task.constraints_matrix, task.b_array)
print(task.constraints_matrix)
# print(task.func)
task.set_start_values()
# print(task.simplex_table)
# print(task.optimal_func)
matrix = np.array([[1.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0 ,1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0, 1.0, 0.0], [1.0, 2.0, 0.0, 0.0, 0.0, 1.0]])

col = task.choose_solving_column()
row, val = task.choose_solving_row(col)

print(task.recount(matrix, col, row))


# task.recount_matrix(col)
# task.recount_optimal(col, val, row)
# task.recount_simplex_table(col, row)
# task.change_func_coefs(col, row)

# col = task.choose_solving_column()
# row, val = task.choose_solving_row(col)
# task.recount_matrix(col)
# print(task.constraints_matrix, task.b_array)
# task.recount_optimal(col, val, row)
# task.recount_simplex_table(col, row)
# task.change_func_coefs(col, row)

[[1 0 1 0 0 0]
 [0 1 0 1 0 0]
 [1 1 0 0 1 0]
 [1 2 0 0 0 1]]
[[1. 0. 1. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0.]
 [1. 1. 0. 0. 1. 0.]
 [1. 2. 0. 0. 0. 1.]]


In [17]:
import json

with open('data.json') as f:
    data = json.load(f)

def f(x):
    return [x[i] * data[' f '][i] for i in range(len(x))]

goal = data[' goal ']

constraints = []
for section in data[' constraints ']:
    constraints.append(dict(section))

print(constraints)

[{' coefs ': [1, 0, 0], ' type ': ' eq ', ' b ': 1}, {' coefs ': [1, 1, 0], ' type ': ' gte ', ' b ': 2}, {' coefs ': [1, 1, 1], ' type ': ' lte ', ' b ': 3}]
